## Imports

In [68]:
# imports
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
from scipy.stats import norm, skew
from scipy.stats import shapiro
import pylab
plt.style.use('ggplot')
from scipy.stats.mstats import winsorize
import time

In [23]:
# input data
ames_data = pd.read_csv('Ames_data.csv')

dat_file = 'project1_testIDs.dat'
test_ids = np.loadtxt(dat_file).T

## Preprocess data

In [63]:
# helper functions
def get_df_info(input_of):
    df_info = pd.DataFrame(columns=['column', 'Null Count', 'Data Type'])
    for col in input_of:
        Null_count  = sum(pd.isnull(input_of[col]))
        dtype = input_of[col].dtype
        df_info = df_info.append({'column': col, 'Null Count': Null_count, 'Data Type': dtype},
                                ignore_index = True)
    
    return df_info

In [64]:
# pre-process train data

def select_column_list(X, target):
    dataframe = X
    dataframe['Sale_Price'] = target
    # Selected Categorical Variables: 
    selected_cat_col = ['Sale_Type', 'Fireplace_Qu', 'Kitchen_Qual', 'Central_Air', 'Heating_QC', 'Bsmt_Exposure', 
                         'Bsmt_Qual', 'Exter_Qual', 'Overall_Qual', 'Neighborhood', 'MS_Zoning', 'Garage_Type', 'Sale_Condition', 
                         'Paved_Drive', 'Garage_Finish', 'MS_SubClass', 'Electrical','Foundation', 'condition_1']
    
    # select numerical variables based on correlation
    df_info = get_df_info(dataframe)
    num_vars = df_info[df_info['Data Type'] != 'object']
    ames_num_cols_df = dataframe.loc[:, dataframe.columns.isin(num_vars.column.tolist())]
    corr_df = ames_num_cols_df.corr()
    corr_df_shortlisted = corr_df.loc[(corr_df['Sale_Price'] > 0.3) | (corr_df['Sale_Price'] < -0.3),:]
    corr_df_shortlisted = corr_df_shortlisted.loc[:, corr_df_shortlisted.columns.isin(corr_df_shortlisted.index)]
    corr_df_shortlisted = corr_df_shortlisted.loc[~corr_df_shortlisted.index.isin(['Garage_Yr_Blt']), ~corr_df_shortlisted.columns.isin(['Garage_Yr_Blt'])]
    corr_df_shortlisted = corr_df_shortlisted.loc[~corr_df_shortlisted.index.isin(['Garage_Area', 'First_Flr_SF']), ~corr_df_shortlisted.columns.isin(['Garage_Area', 'First_Flr_SF'])]
    
    # finalize columns selected
    selected_cat_col.extend(corr_df_shortlisted.columns) 
    
    return selected_cat_col

def transform_xtrain(xtrn, ytrn):
    
    # winsorize outliers in Gr_Liv_Area
    winsz_Gr_Liv_Area = winsorize(xtrn.Gr_Liv_Area, limits=[0.05, 0.05])
    xtrn['Gr_Liv_Area'] = winsz_Gr_Liv_Area
    
    winsz_sale_price = winsorize(ytrn, limits=[0.05, 0.05])
    ytrn = winsz_sale_price

    # drop PID since this won't affect Sale_Price
    xtrn.drop(['PID'], axis=1, inplace=True)
    
    # Convert Year values to age and drop unnecessary columns
    xtrn.drop(['Year_Remod_Add', 'Mo_Sold', 'Year_Sold'], axis=1, inplace=True)
    xtrn['Age_of_Property'] = 2010 - xtrn['Year_Built']
    xtrn.drop(['Year_Built'], axis=1, inplace=True)                                            
    
    selected_cat_col = select_column_list(xtrn, ytrn)
    df_final = xtrn.loc[:, xtrn.columns.isin(selected_cat_col)]
    
    return df_final

def transform_ytrain(ytrn):
    winsz_sale_price = winsorize(ytrn, limits=[0.05, 0.05])
    ytrn = winsz_sale_price
    
    return ytrn


def transform_xtest(xtrn, ytrn, xtst, ytst):
    
    # get winsorized limits in Gr_Liv_Area
    winsz_Gr_Liv_Area = winsorize(xtrn.Gr_Liv_Area, limits=[0.05, 0.05])
    a = np.array(xtst['Gr_Liv_Area'].values.tolist())
    xtst['Gr_Liv_Area'] = np.where(a > max(winsz_Gr_Liv_Area), max(winsz_Gr_Liv_Area), a).tolist()
    xtst['Gr_Liv_Area'] = np.where(a < min(winsz_Gr_Liv_Area), min(winsz_Gr_Liv_Area), a).tolist()
    
    # get winsorized limits in Sale_Price
    winsz_sale_price = winsorize(ytrn, limits=[0.05, 0.05])
    a = np.array(ytst.values.tolist())
    ytst = np.where(a > max(winsz_sale_price), max(winsz_sale_price), a).tolist()
    ytst = np.where(a < min(winsz_sale_price), min(winsz_sale_price), a).tolist()

    # drop PID since this won't affect Sale_Price
    xtst.drop(['PID'], axis=1, inplace=True)
    
    # Convert Year values to age and drop unnecessary columns
    xtst.drop(['Year_Remod_Add', 'Mo_Sold', 'Year_Sold'], axis=1, inplace=True)
    xtst['Age_of_Property'] = 2010 - xtst['Year_Built']
    xtst.drop(['Year_Built'], axis=1, inplace=True)
    
    selected_cat_col = select_column_list(xtrn, ytrn)
    xtst_final = xtst.loc[:, xtst.columns.isin(selected_cat_col)]
    
    return xtst_final

def transform_ytest(ytst, ytrn):
    # get winsorized limits in Sale_Price
    winsz_sale_price = winsorize(ytrn, limits=[0.05, 0.05])
    a = np.array(ytst.values.tolist())
    ytst = np.where(a > max(winsz_sale_price), max(winsz_sale_price), a).tolist()
    ytst = np.where(a < min(winsz_sale_price), min(winsz_sale_price), a).tolist()
    
    return ytst

In [66]:
from sklearn import preprocessing
lbl = preprocessing.LabelEncoder()
X = ames_data.loc[:, ~ames_data.columns.str.contains('Sale_Price')]

j = 2
X_train = X.iloc[~X.index.isin(test_ids[j]),:]
X_test = X.iloc[X.index.isin(test_ids[j]),:]
y_train = ames_data.iloc[~ames_data.index.isin(test_ids[j]),:]['Sale_Price']
y_test = ames_data.iloc[ames_data.index.isin(test_ids[j]),:]['Sale_Price']

In [67]:
xtrn_transformed = transform_xtest(X_train, y_train, X_test, y_test)
xtst_transformed = transform_xtrain(X_train, y_train)
ytrn_transformed = transform_ytrain(y_train)
ytst_transformed = transform_ytest(y_test, y_train)

## XgBoost Hyperparameter tuning

In [74]:
time.time()

1634594751.0137324